# Char RNN

In this notebook, we will go through basics of Char-RNN and use different datasets to create different RNNs.

Here we will use [PyTorch](http://pytorch.org/tutorials/  "PyTorch Tutorial").


Hey yo, but how to see what CNN sees?

Everything is explained in-detail in [blog post](). This is notebook which replicates the result of blog and runs in colab. Enjoy!


#### Run in Colab

You can run this notebook in google colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dudeperf3ct/DL_notebooks/blob/master/RNN/char_rnn_pytorch.ipynb)




Here are some other very interesting results, [Cooking-Recipe](https://gist.github.com/nylki/1efbaa36635956d35bcc), [Obama-RNN](https://medium.com/@samim/obama-rnn-machine-generated-political-speeches-c8abd18a2ea0), [Bible-RNN](https://twitter.com/RNN_Bible), [Folk-music](https://soundcloud.com/seaandsailor/sets/char-rnn-composes-irish-folk-music), [Learning Holiness](https://cpury.github.io/learning-holiness/), [AI Weirdness](http://aiweirdness.com/), [Auto-Generating Clickbait](https://larseidnes.com/2015/10/13/auto-generating-clickbait-with-recurrent-neural-networks/).

## Download data

In [0]:
! wget "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt" -P {'data/'}
! wget "https://s3.amazonaws.com/text-datasets/nietzsche.txt" -P {'data/'}
! wget "http://www.gutenberg.org/files/31100/31100.txt" -P {'data/'}
! wget "http://www.gutenberg.org/cache/epub/29765/pg29765.txt" -P {'data/'}
! wget "https://raw.githubusercontent.com/ryanmcdermott/trump-speeches/master/speeches.txt" -P {'data/'}
! wget "https://raw.githubusercontent.com/mcleonard/pytorch-charRNN/master/data/anna.txt" -P {'data/'}
! wget "https://raw.githubusercontent.com/samim23/obama-rnn/master/input.txt" -P {'data/obama/'}

--2019-02-17 10:28:25--  https://raw.githubusercontent.com/samim23/obama-rnn/master/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4250014 (4.1M) [text/plain]
Saving to: ‘data/obama/input.txt’

input.txt           100%[===================>]   4.05M  --.-KB/s    in 0.1s    

2019-02-17 10:28:25 (34.2 MB/s) - ‘data/obama/input.txt’ saved [4250014/4250014]



In [0]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print (device)

cuda:0


### Enchancements

To further improve the results, here are some things that can be done:

1. Instead of converting all input text to lower we can use capitilization as it is.

2. To use all data, we could create batch generator to feed all batches instead of loading everything on RAM

## ShakespeareRNN

In [0]:
file_path = 'data/input.txt'

with open(file_path, 'r') as f:
  
    data = f.read()
    
print ('First 200 characters of data\n')
print (data[:200])

First 200 characters of data

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [0]:
# unique characters in dataset and its count
chars = set(data)
vocab_size = len(set(data))
data_size = len(data)
print ('Unique characters:', chars)
print ('Length of Unique characters:', vocab_size)
print ('Number of characters in data:', data_size)

#restricting so that we can load all data onto RAM
#uncomment this line to run on all data
data = data[:515394]
data_size = len(data)

Unique characters: {'C', 'a', 'S', 'T', 'm', '.', 'Y', 'x', 'K', 'b', 'h', 'z', 'w', 's', 'k', 'M', 'L', 'N', 't', 'R', '&', 'Z', 'D', 'q', 'F', ' ', 'd', 'O', 'u', 'j', '-', 'I', ':', 'A', 'c', 'g', 'l', 'Q', 'i', 'e', 'E', 'B', 'y', "'", 'f', '?', 'n', ',', '!', 'P', ';', 'X', 'J', '3', 'V', 'r', 'W', 'p', 'H', '$', 'G', 'v', 'U', '\n', 'o'}
Length of Unique characters: 65
Number of characters in data: 1115394


In [0]:
char2id = {ch:i for i, ch in enumerate(chars)}
id2char = {i:ch for i, ch in enumerate(chars)}

print ('Characters to id\n')
print (char2id)

Characters to id

{'C': 0, 'a': 1, 'S': 2, 'T': 3, 'm': 4, '.': 5, 'Y': 6, 'x': 7, 'K': 8, 'b': 9, 'h': 10, 'z': 11, 'w': 12, 's': 13, 'k': 14, 'M': 15, 'L': 16, 'N': 17, 't': 18, 'R': 19, '&': 20, 'Z': 21, 'D': 22, 'q': 23, 'F': 24, ' ': 25, 'd': 26, 'O': 27, 'u': 28, 'j': 29, '-': 30, 'I': 31, ':': 32, 'A': 33, 'c': 34, 'g': 35, 'l': 36, 'Q': 37, 'i': 38, 'e': 39, 'E': 40, 'B': 41, 'y': 42, "'": 43, 'f': 44, '?': 45, 'n': 46, ',': 47, '!': 48, 'P': 49, ';': 50, 'X': 51, 'J': 52, '3': 53, 'V': 54, 'r': 55, 'W': 56, 'p': 57, 'H': 58, '$': 59, 'G': 60, 'v': 61, 'U': 62, '\n': 63, 'o': 64}


In [0]:
# cut the text into fixed size inputs of length maxlen
maxlen = 100
sentences = []
next_chars = []

end = data_size - maxlen
for i in range(0, end, maxlen):
    sentences.append([char2id[ch] for ch in data[i : i+maxlen]])
    next_chars.append([char2id[ch] for ch in data[i+1: i+maxlen+1]])

In [0]:
print (sentences[0], len(sentences[0]), len(sentences))
print (next_chars[0], len(next_chars[0]), len(next_chars))

[24, 38, 55, 13, 18, 25, 0, 38, 18, 38, 11, 39, 46, 32, 63, 41, 39, 44, 64, 55, 39, 25, 12, 39, 25, 57, 55, 64, 34, 39, 39, 26, 25, 1, 46, 42, 25, 44, 28, 55, 18, 10, 39, 55, 47, 25, 10, 39, 1, 55, 25, 4, 39, 25, 13, 57, 39, 1, 14, 5, 63, 63, 33, 36, 36, 32, 63, 2, 57, 39, 1, 14, 47, 25, 13, 57, 39, 1, 14, 5, 63, 63, 24, 38, 55, 13, 18, 25, 0, 38, 18, 38, 11, 39, 46, 32, 63, 6, 64, 28] 100 5153
[38, 55, 13, 18, 25, 0, 38, 18, 38, 11, 39, 46, 32, 63, 41, 39, 44, 64, 55, 39, 25, 12, 39, 25, 57, 55, 64, 34, 39, 39, 26, 25, 1, 46, 42, 25, 44, 28, 55, 18, 10, 39, 55, 47, 25, 10, 39, 1, 55, 25, 4, 39, 25, 13, 57, 39, 1, 14, 5, 63, 63, 33, 36, 36, 32, 63, 2, 57, 39, 1, 14, 47, 25, 13, 57, 39, 1, 14, 5, 63, 63, 24, 38, 55, 13, 18, 25, 0, 38, 18, 38, 11, 39, 46, 32, 63, 6, 64, 28, 25] 100 5153


In [0]:
print ('Input:', ''.join(id2char[i] for i in sentences[0]))
print ('Output:', ''.join(id2char[i] for i in next_chars[0]))

Input: First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
Output: irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


In [0]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='float32')[y]

In [0]:
X = np.zeros((len(sentences), maxlen, vocab_size), dtype=np.float32)
one_hot = [to_categorical(c, num_classes=vocab_size) for i in range(len(sentences)) for c in sentences[i]]
X = np.array(one_hot).reshape(X.shape)
print (X[0], X.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]] (5153, 100, 65)


In [0]:
Y = np.zeros((len(next_chars), maxlen), dtype=np.float32)
Y = np.array(next_chars).reshape(Y.shape)
print (Y[0], Y.shape)

[38 55 13 18 25  0 38 18 38 11 39 46 32 63 41 39 44 64 55 39 25 12 39 25
 57 55 64 34 39 39 26 25  1 46 42 25 44 28 55 18 10 39 55 47 25 10 39  1
 55 25  4 39 25 13 57 39  1 14  5 63 63 33 36 36 32 63  2 57 39  1 14 47
 25 13 57 39  1 14  5 63 63 24 38 55 13 18 25  0 38 18 38 11 39 46 32 63
  6 64 28 25] (5153, 100)


In [0]:
train_x, val_x, train_y, val_y = train_test_split(X, Y, test_size=0.05)
print ('Training:', train_x.shape, train_y.shape)
print ('Validation:', val_x.shape, val_y.shape)

Training: (4895, 100, 65) (4895, 100)
Validation: (258, 100, 65) (258, 100)


## CharRNN PyTorch Model

Code adapted from : [Link](https://github.com/mcleonard/pytorch-charRNN/blob/master/TorchRNN.ipynb)

In [0]:
class CharRNN(nn.Module):
  
    def __init__(self, batch_size=128, n_hidden=512, n_layers=2, drop_prob=0.5):
      
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
      
        self.dropout = nn.Dropout(drop_prob)
        self.lstm = nn.LSTM(vocab_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        self.fc = nn.Linear(n_hidden, vocab_size)
        
        self.init_weights()
        
    def forward(self, x, hc):
        ''' Forward pass through the network '''
        
        x, (h, c) = self.lstm(x, hc)
        x = self.dropout(x)
        
        # Stack up LSTM outputs
        x = x.view(x.size()[0]*x.size()[1], self.n_hidden)
        
        x = self.fc(x)
        
        return x, (h, c)
    
    def predict(self, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
        
            Returns the predicted character and the hidden state.
        '''
        
        if h is None:
            h = self.init_hidden(1)
        
        x = np.array([[char2id[char]]])
        x = to_categorical(x, num_classes=vocab_size)
        
        with torch.no_grad():
            h = tuple([Variable(each.data) for each in h])
            inputs = Variable(torch.from_numpy(x))
        inputs = inputs.to(device)
        
        out, h = self.forward(inputs, h)
        p = F.softmax(out).data
        p = p.to(device)
        
        if top_k is None:
            top_ch = np.arange(vocab_size)
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.cpu().numpy().squeeze()
        
        p = p.cpu().numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
            
        return id2char[char], h
    
    def init_weights(self):
        ''' Initialize weights for fully connected layer '''
      
        # Set bias tensor to all zeros
        self.fc.bias.data.fill_(0)
        # FC weights as random uniform
        self.fc.weight.data.uniform_(-1, 1)
        
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return (Variable(weight.new(self.n_layers, batch_size, self.n_hidden).zero_()),
                Variable(weight.new(self.n_layers, batch_size, self.n_hidden).zero_()))

In [0]:
net = CharRNN(n_hidden=512, n_layers=2)
net.to(device)

CharRNN(
  (dropout): Dropout(p=0.5)
  (lstm): LSTM(65, 512, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=512, out_features=65, bias=True)
)

In [0]:
opt = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [0]:
def train(net, train_x, train_y, val_x, val_y, opt, criterion, epochs=25, batch_size=128, maxlen=100, clip=5, print_every=10):
    
    net.train()
    counter = 0
    
    for e in range(epochs):
        
        h = net.init_hidden(batch_size)
        
        for k in range(0, train_x.shape[0]-batch_size, batch_size):
            
            counter += 1
            # batch data and convert to torch tensors
            x, y = train_x[k : k+batch_size], train_y[k : k+batch_size]
            x, y = torch.from_numpy(x), torch.from_numpy(y)
            
            
            inputs, targets = Variable(x), Variable(y)
            inputs, targets = inputs.to(device), targets.to(device)
            #print (counter, inputs.shape, targets.shape)
            
            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([Variable(each.data) for each in h])

            net.zero_grad()
            output, h = net.forward(inputs, h)
            loss = criterion(output, targets.view(batch_size*maxlen))
            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            opt.step()
            
            if counter % print_every == 0:
                
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                
                for k in range(0, val_x.shape[0]-batch_size, batch_size):
                  
                    # One-hot encode our data and make them Torch tensors
                    x, y = val_x[k : k+batch_size], val_y[k : k+batch_size]
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
            
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    with torch.no_grad():
                        val_h = tuple([Variable(each.data) for each in val_h])
                        inputs, targets = Variable(x), Variable(y)
                    
                    inputs, targets = inputs.to(device), targets.to(device)

                    output, val_h = net.forward(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*maxlen))
                
                    val_losses.append(val_loss.item())
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [0]:
train(net, train_x, train_y, val_x, val_y, opt=opt, criterion=criterion, epochs=25,
      batch_size=128, maxlen=100, clip=5, print_every=10)

Epoch: 1/25... Step: 10... Loss: 3.4232... Val Loss: 3.4138
Epoch: 1/25... Step: 20... Loss: 3.3019... Val Loss: 3.2861
Epoch: 1/25... Step: 30... Loss: 3.1572... Val Loss: 3.1254
Epoch: 2/25... Step: 40... Loss: 2.9588... Val Loss: 2.9408
Epoch: 2/25... Step: 50... Loss: 2.7755... Val Loss: 2.7430
Epoch: 2/25... Step: 60... Loss: 2.6318... Val Loss: 2.6211
Epoch: 2/25... Step: 70... Loss: 2.5555... Val Loss: 2.5357
Epoch: 3/25... Step: 80... Loss: 2.4869... Val Loss: 2.4701
Epoch: 3/25... Step: 90... Loss: 2.4321... Val Loss: 2.4182
Epoch: 3/25... Step: 100... Loss: 2.3904... Val Loss: 2.3831
Epoch: 3/25... Step: 110... Loss: 2.3869... Val Loss: 2.3444
Epoch: 4/25... Step: 120... Loss: 2.3362... Val Loss: 2.3079
Epoch: 4/25... Step: 130... Loss: 2.3136... Val Loss: 2.2781
Epoch: 4/25... Step: 140... Loss: 2.2792... Val Loss: 2.2485
Epoch: 4/25... Step: 150... Loss: 2.2742... Val Loss: 2.2318
Epoch: 5/25... Step: 160... Loss: 2.2793... Val Loss: 2.2047
Epoch: 5/25... Step: 170... Loss:

In [0]:
def sample(net, size, prime='The', top_k=None):

    net.eval()
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = net.predict(ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = net.predict(chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [0]:
print(sample(net, 2000, prime='To be or not to be', top_k=5))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


To be or not to bell,
And made hus a polere of my hand.

CORIOLANUS:
I have be a there is shome to me,
Well sheep had stain and shanger of a morth.

SICINIUS:
The such one state a childry, wherefore.

MENENIUS:
O worthy hands,
The stroves of the son, time out to my stears on a man armon and wifold to hear hus that a stranges, who, the whare as he to me to he to me that tell thee,
To see this bands of theing of a shripts and whom his sonsering with a store as was a solfor our thee?

Second Servingman:
Which he shallst an hally the strieges of subres of the cause, and thy barther of the chombers, breath my brow to tell thee to me, and this dause this his some himself so men,
The secomair that a wenter's sides are as him as
this and to see it hat.

BRUTUS:
With the so farst wise high this freens,
But that with hapet heart the tales and have
The sone of make this sour are, this the man much anse
And which the partinious shall of a goneren sents,
Which the word wind they shall a place they 

## NietzscheRNN

In [0]:
file_path = 'data/nietzsche.txt'

with open(file_path, 'r') as f:
  
    data = f.read()
    
print ('First 200 characters of data\n')
print (data[:200])

First 200 characters of data

PREFACE


SUPPOSING that Truth is a woman--what then? Is there not ground
for suspecting that all philosophers, in so far as they have been
dogmatists, have failed to understand women--that the terrib


In [0]:
# unique characters in dataset and its count
chars = set(data)
vocab_size = len(set(data))
data_size = len(data)
print ('Unique characters:', chars)
print ('Length of Unique characters:', vocab_size)
print ('Number of characters in data:', data_size)

#restricting so that we can load all data onto RAM
#uncomment this line to run on all data
data = data[:200893]
data_size = len(data)

Unique characters: {'D', 'p', 'm', 'R', 'v', 'I', ';', '8', 'l', 'e', '_', '=', 'a', 'F', 'd', '7', 'Æ', 'j', 'X', '[', ')', 'G', 'B', '1', 'æ', 'g', 'Z', ' ', '5', 'S', 'U', 'c', 'V', 'x', '0', 'ë', 's', 'Y', ',', '?', 'k', '-', 'é', 'O', 'q', 'P', 'T', 'N', 'ä', 'o', '.', 'h', '\n', 'z', "'", '3', 'K', 'H', '!', '9', 'C', 'u', 'J', 'n', 'E', 'f', '4', ']', 'A', 'w', 'r', 'W', 't', 'Q', 'L', ':', 'i', 'b', '(', 'M', '2', '6', '"', 'y'}
Length of Unique characters: 84
Number of characters in data: 600893


In [0]:
char2id = {ch:i for i, ch in enumerate(chars)}
id2char = {i:ch for i, ch in enumerate(chars)}

print ('Characters to id\n')
print (char2id)

Characters to id

{'D': 0, 'p': 1, 'm': 2, 'R': 3, 'v': 4, 'I': 5, ';': 6, '8': 7, 'l': 8, 'e': 9, '_': 10, '=': 11, 'a': 12, 'F': 13, 'd': 14, '7': 15, 'Æ': 16, 'j': 17, 'X': 18, '[': 19, ')': 20, 'G': 21, 'B': 22, '1': 23, 'æ': 24, 'g': 25, 'Z': 26, ' ': 27, '5': 28, 'S': 29, 'U': 30, 'c': 31, 'V': 32, 'x': 33, '0': 34, 'ë': 35, 's': 36, 'Y': 37, ',': 38, '?': 39, 'k': 40, '-': 41, 'é': 42, 'O': 43, 'q': 44, 'P': 45, 'T': 46, 'N': 47, 'ä': 48, 'o': 49, '.': 50, 'h': 51, '\n': 52, 'z': 53, "'": 54, '3': 55, 'K': 56, 'H': 57, '!': 58, '9': 59, 'C': 60, 'u': 61, 'J': 62, 'n': 63, 'E': 64, 'f': 65, '4': 66, ']': 67, 'A': 68, 'w': 69, 'r': 70, 'W': 71, 't': 72, 'Q': 73, 'L': 74, ':': 75, 'i': 76, 'b': 77, '(': 78, 'M': 79, '2': 80, '6': 81, '"': 82, 'y': 83}


In [0]:
# cut the text into fixed size inputs of length maxlen
maxlen = 100
sentences = []
next_chars = []

end = data_size - maxlen
for i in range(0, end, maxlen):
    sentences.append([char2id[ch] for ch in data[i : i+maxlen]])
    next_chars.append([char2id[ch] for ch in data[i+1: i+maxlen+1]])

In [0]:
print (sentences[0], len(sentences[0]), len(sentences))
print (next_chars[0], len(next_chars[0]), len(next_chars))

[45, 3, 64, 13, 68, 60, 64, 52, 52, 52, 29, 30, 45, 45, 43, 29, 5, 47, 21, 27, 72, 51, 12, 72, 27, 46, 70, 61, 72, 51, 27, 76, 36, 27, 12, 27, 69, 49, 2, 12, 63, 41, 41, 69, 51, 12, 72, 27, 72, 51, 9, 63, 39, 27, 5, 36, 27, 72, 51, 9, 70, 9, 27, 63, 49, 72, 27, 25, 70, 49, 61, 63, 14, 52, 65, 49, 70, 27, 36, 61, 36, 1, 9, 31, 72, 76, 63, 25, 27, 72, 51, 12, 72, 27, 12, 8, 8, 27, 1, 51] 100 2008
[3, 64, 13, 68, 60, 64, 52, 52, 52, 29, 30, 45, 45, 43, 29, 5, 47, 21, 27, 72, 51, 12, 72, 27, 46, 70, 61, 72, 51, 27, 76, 36, 27, 12, 27, 69, 49, 2, 12, 63, 41, 41, 69, 51, 12, 72, 27, 72, 51, 9, 63, 39, 27, 5, 36, 27, 72, 51, 9, 70, 9, 27, 63, 49, 72, 27, 25, 70, 49, 61, 63, 14, 52, 65, 49, 70, 27, 36, 61, 36, 1, 9, 31, 72, 76, 63, 25, 27, 72, 51, 12, 72, 27, 12, 8, 8, 27, 1, 51, 76] 100 2008


In [0]:
print ('Input:', ''.join(id2char[i] for i in sentences[0]))
print ('Output:', ''.join(id2char[i] for i in next_chars[0]))

Input: PREFACE


SUPPOSING that Truth is a woman--what then? Is there not ground
for suspecting that all ph
Output: REFACE


SUPPOSING that Truth is a woman--what then? Is there not ground
for suspecting that all phi


In [0]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='float32')[y]

In [0]:
X = np.zeros((len(sentences), maxlen, vocab_size), dtype=np.float32)
one_hot = [to_categorical(c, num_classes=vocab_size) for i in range(len(sentences)) for c in sentences[i]]
X = np.array(one_hot).reshape(X.shape)
print (X[0], X.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (2008, 100, 84)


In [0]:
Y = np.zeros((len(next_chars), maxlen), dtype=np.float32)
Y = np.array(next_chars).reshape(Y.shape)
print (Y[0], Y.shape)

[ 3 64 13 68 60 64 52 52 52 29 30 45 45 43 29  5 47 21 27 72 51 12 72 27
 46 70 61 72 51 27 76 36 27 12 27 69 49  2 12 63 41 41 69 51 12 72 27 72
 51  9 63 39 27  5 36 27 72 51  9 70  9 27 63 49 72 27 25 70 49 61 63 14
 52 65 49 70 27 36 61 36  1  9 31 72 76 63 25 27 72 51 12 72 27 12  8  8
 27  1 51 76] (2008, 100)


In [0]:
train_x, val_x, train_y, val_y = train_test_split(X, Y, test_size=0.2)
print ('Training:', train_x.shape, train_y.shape)
print ('Validation:', val_x.shape, val_y.shape)

Training: (1606, 100, 84) (1606, 100)
Validation: (402, 100, 84) (402, 100)


In [0]:
net = CharRNN(n_hidden=512, n_layers=2)
net.to(device)

CharRNN(
  (dropout): Dropout(p=0.5)
  (lstm): LSTM(84, 512, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=512, out_features=84, bias=True)
)

In [0]:
opt = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [0]:
train(net, train_x, train_y, val_x, val_y, opt=opt, criterion=criterion, epochs=25,
      batch_size=128, maxlen=100, clip=5, print_every=10)

Epoch: 1/25... Step: 10... Loss: 3.3786... Val Loss: 3.3212
Epoch: 2/25... Step: 20... Loss: 3.2585... Val Loss: 3.2214
Epoch: 3/25... Step: 30... Loss: 3.1298... Val Loss: 3.0956
Epoch: 4/25... Step: 40... Loss: 2.9663... Val Loss: 2.9281
Epoch: 5/25... Step: 50... Loss: 2.7799... Val Loss: 2.7638
Epoch: 5/25... Step: 60... Loss: 2.6517... Val Loss: 2.6701
Epoch: 6/25... Step: 70... Loss: 2.6086... Val Loss: 2.6001
Epoch: 7/25... Step: 80... Loss: 2.5806... Val Loss: 2.5509
Epoch: 8/25... Step: 90... Loss: 2.5206... Val Loss: 2.5102
Epoch: 9/25... Step: 100... Loss: 2.4815... Val Loss: 2.4703
Epoch: 10/25... Step: 110... Loss: 2.4498... Val Loss: 2.4448
Epoch: 10/25... Step: 120... Loss: 2.3832... Val Loss: 2.4092
Epoch: 11/25... Step: 130... Loss: 2.3723... Val Loss: 2.3779
Epoch: 12/25... Step: 140... Loss: 2.3804... Val Loss: 2.3525
Epoch: 13/25... Step: 150... Loss: 2.3165... Val Loss: 2.3275
Epoch: 14/25... Step: 160... Loss: 2.3002... Val Loss: 2.2984
Epoch: 15/25... Step: 170..

In [0]:
print(sample(net, 2000, prime='Every great philosophy', top_k=5))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Every great philosophy.

65. Wo lowe as all sere that the prost of the his oncation. Ane the
plessition of thisk of the perition of serecasing that at the porest of the calute a perisis to the sachite of this sears fart alo all trisg of a thish on is ancerting, and and
touth of--in this surablicising tion and
that in the concauly
of to sente them trigh to be a dentired of their have in the riscess, itself the sained and
mosters and as ont ofle the mort of the moderation of shaig ance tain this sears fict of that
ther in is alles and efont and
much is the resting or one
to
their incaine, and insucalie at of the sarn and thus a manting this sain for and inserites,
whe inselies itself, indorstanced. To all the conciest of
muralined which is incintly te ant intoring to and ast that the
pertert of such as astominated to be tree the sare imself camself in onlereds of cersingicare one ore penseste and surition ancestand
tomestite of a surition the man to that he priles in the rost as munterst

## AustenRNN

In [0]:
file_path = 'data/31100.txt'
import io

with io.open(file_path, 'r', encoding='windows-1252') as f:
  
    data = f.read().lower()
    
print ('First 200 characters of data\n')
print (data[:200])

First 200 characters of data


project gutenberg's the complete works of jane austen, by jane austen

this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  you may copy it, give it aw


In [0]:
# unique characters in dataset and its count
chars = set(data)
vocab_size = len(set(data))
data_size = len(data)
print ('Unique characters:', chars)
print ('Length of Unique characters:', vocab_size)
print ('Number of characters in data:', data_size)

#restricting so that we can load all data onto RAM
#uncomment this line to run on all data
data = data[:473597]
data_size = len(data)

Unique characters: {',', 'g', ']', '.', '(', '5', '1', '?', 's', 'v', 'j', 'o', '/', 'i', 'k', '\n', '%', '&', 'm', '[', 'l', '4', 'r', ' ', 'y', '\xa0', ';', 'q', 'u', '-', 'f', ':', '6', '9', 'b', '0', 'a', 'h', ')', 'p', '!', 'c', '#', '_', '$', "'", 'd', '8', '*', '2', '"', 'n', '3', 'e', 't', '7', 'x', 'z', '@', 'w'}
Length of Unique characters: 60
Number of characters in data: 4373597


In [0]:
char2id = {ch:i for i, ch in enumerate(chars)}
id2char = {i:ch for i, ch in enumerate(chars)}

print ('Characters to id\n')
print (char2id)

Characters to id

{',': 0, 'g': 1, ']': 2, '.': 3, '(': 4, '5': 5, '1': 6, '?': 7, 's': 8, 'v': 9, 'j': 10, 'o': 11, '/': 12, 'i': 13, 'k': 14, '\n': 15, '%': 16, '&': 17, 'm': 18, '[': 19, 'l': 20, '4': 21, 'r': 22, ' ': 23, 'y': 24, '\xa0': 25, ';': 26, 'q': 27, 'u': 28, '-': 29, 'f': 30, ':': 31, '6': 32, '9': 33, 'b': 34, '0': 35, 'a': 36, 'h': 37, ')': 38, 'p': 39, '!': 40, 'c': 41, '#': 42, '_': 43, '$': 44, "'": 45, 'd': 46, '8': 47, '*': 48, '2': 49, '"': 50, 'n': 51, '3': 52, 'e': 53, 't': 54, '7': 55, 'x': 56, 'z': 57, '@': 58, 'w': 59}


In [0]:
# cut the text into fixed size inputs of length maxlen
maxlen = 100
sentences = []
next_chars = []

end = data_size - maxlen
for i in range(0, end, maxlen):
    sentences.append([char2id[ch] for ch in data[i : i+maxlen]])
    next_chars.append([char2id[ch] for ch in data[i+1: i+maxlen+1]])

In [0]:
print (sentences[0], len(sentences[0]), len(sentences))
print (next_chars[0], len(next_chars[0]), len(next_chars))

[15, 39, 22, 11, 10, 53, 41, 54, 23, 1, 28, 54, 53, 51, 34, 53, 22, 1, 45, 8, 23, 54, 37, 53, 23, 41, 11, 18, 39, 20, 53, 54, 53, 23, 59, 11, 22, 14, 8, 23, 11, 30, 23, 10, 36, 51, 53, 23, 36, 28, 8, 54, 53, 51, 0, 23, 34, 24, 23, 10, 36, 51, 53, 23, 36, 28, 8, 54, 53, 51, 15, 15, 54, 37, 13, 8, 23, 53, 34, 11, 11, 14, 23, 13, 8, 23, 30, 11, 22, 23, 54, 37, 53, 23, 28, 8, 53, 23, 11, 30] 100 4735
[39, 22, 11, 10, 53, 41, 54, 23, 1, 28, 54, 53, 51, 34, 53, 22, 1, 45, 8, 23, 54, 37, 53, 23, 41, 11, 18, 39, 20, 53, 54, 53, 23, 59, 11, 22, 14, 8, 23, 11, 30, 23, 10, 36, 51, 53, 23, 36, 28, 8, 54, 53, 51, 0, 23, 34, 24, 23, 10, 36, 51, 53, 23, 36, 28, 8, 54, 53, 51, 15, 15, 54, 37, 13, 8, 23, 53, 34, 11, 11, 14, 23, 13, 8, 23, 30, 11, 22, 23, 54, 37, 53, 23, 28, 8, 53, 23, 11, 30, 23] 100 4735


In [0]:
print ('Input:', ''.join(id2char[i] for i in sentences[0]))
print ('Output:', ''.join(id2char[i] for i in next_chars[0]))

Input: 
project gutenberg's the complete works of jane austen, by jane austen

this ebook is for the use of
Output: project gutenberg's the complete works of jane austen, by jane austen

this ebook is for the use of 


In [0]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='float32')[y]

In [0]:
X = np.zeros((len(sentences), maxlen, vocab_size), dtype=np.float32)
one_hot = [to_categorical(c, num_classes=vocab_size) for i in range(len(sentences)) for c in sentences[i]]
X = np.array(one_hot).reshape(X.shape)
print (X[0], X.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (4735, 100, 60)


In [0]:
Y = np.zeros((len(next_chars), maxlen), dtype=np.float32)
Y = np.array(next_chars).reshape(Y.shape)
print (Y[0], Y.shape)

[39 22 11 10 53 41 54 23  1 28 54 53 51 34 53 22  1 45  8 23 54 37 53 23
 41 11 18 39 20 53 54 53 23 59 11 22 14  8 23 11 30 23 10 36 51 53 23 36
 28  8 54 53 51  0 23 34 24 23 10 36 51 53 23 36 28  8 54 53 51 15 15 54
 37 13  8 23 53 34 11 11 14 23 13  8 23 30 11 22 23 54 37 53 23 28  8 53
 23 11 30 23] (4735, 100)


In [0]:
train_x, val_x, train_y, val_y = train_test_split(X, Y, test_size=0.2)
print ('Training:', train_x.shape, train_y.shape)
print ('Validation:', val_x.shape, val_y.shape)

Training: (3788, 100, 60) (3788, 100)
Validation: (947, 100, 60) (947, 100)


In [0]:
net = CharRNN(n_hidden=512, n_layers=2)
net.to(device)

CharRNN(
  (dropout): Dropout(p=0.5)
  (lstm): LSTM(60, 512, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=512, out_features=60, bias=True)
)

In [0]:
opt = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [0]:
train(net, train_x, train_y, val_x, val_y, opt=opt, criterion=criterion, epochs=25,
      batch_size=128, maxlen=100, clip=5, print_every=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch: 1/25... Step: 10... Loss: 3.1571... Val Loss: 3.1588
Epoch: 1/25... Step: 20... Loss: 3.0604... Val Loss: 3.0484
Epoch: 2/25... Step: 30... Loss: 2.9630... Val Loss: 2.9389
Epoch: 2/25... Step: 40... Loss: 2.8130... Val Loss: 2.7876
Epoch: 2/25... Step: 50... Loss: 2.6253... Val Loss: 2.6408
Epoch: 3/25... Step: 60... Loss: 2.5456... Val Loss: 2.5417
Epoch: 3/25... Step: 70... Loss: 2.4814... Val Loss: 2.4742
Epoch: 3/25... Step: 80... Loss: 2.4178... Val Loss: 2.4235
Epoch: 4/25... Step: 90... Loss: 2.3910... Val Loss: 2.3837
Epoch: 4/25... Step: 100... Loss: 2.3445... Val Loss: 2.3465
Epoch: 4/25... Step: 110... Loss: 2.3141... Val Loss: 2.3123
Epoch: 5/25... Step: 120... Loss: 2.2844... Val Loss: 2.2816
Epoch: 5/25... Step: 130... Loss: 2.2444... Val Loss: 2.2545
Epoch: 5/25... Step: 140... Loss: 2.2485... Val Loss: 2.2241
Epoch: 6/25... Step: 150... Loss: 2.1817... Val Loss: 2.1955
Epoch: 6/25... Step: 160... Loss: 2.1732... Val Loss: 2.1694
Epoch: 6/25... Step: 170... Loss:

In [0]:
print(sample(net, 2000, prime='vanity was the beginning', top_k=5))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


vanity was the beginning on one to the father was not believed the mort any our preter to
being and an any of me always
and mrs smother, i she chared as to the farly to stay
sersperd and mind existen if an
a concert was to the parracely of the mind, and at
lyst had been astention with the poor a seeing to be
an to book of the part convensed,
and the seemed to be and the stall, the country, and they were such the friends, or the
conferst the cried to have seen one other say."

"nessed you make something to some a concisions, to be care, i with more them.

anne was too to sen this
at the same sense.

i am sure you, said he seet to be concerned to her, to be the manner of having a gettlemen, was to the pase of she seemed his friends he
had been as
she had been to home than of at his foot, who had a gid not the forture all his acquaintance.  anne some to
decend the rest tearns to have house indeed, it was not and all mr elliot and the parting one of him.

she had
samed her side, and was no

## DictionaryRNN

In [0]:
file_path = 'data/pg29765.txt'

with open(file_path, 'r') as f:
  
    data = f.read().lower()
    
print ('First 200 characters of data\n')
print (data[:200])

First 200 characters of data

﻿the project gutenberg ebook of webster's unabridged dictionary, by various

this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  you may copy it, give 


In [0]:
# unique characters in dataset and its count
chars = set(data)
vocab_size = len(set(data))
data_size = len(data)
print ('Unique characters:', chars)
print ('Length of Unique characters:', vocab_size)
print ('Number of characters in data:', data_size)

#restricting so that we can load all data onto RAM
#uncomment this line to run on all data
data = data[:156206]
data_size = len(data)

Unique characters: {'&', 'a', '|', 'þ', 'h', 'w', '^', 'd', 'â', '1', 'ó', 'ã', 'ê', 'r', '@', '*', 'ë', 'y', '9', 'o', 'f', ' ', '/', 'ï', 'à', '½', 'å', '¼', 'k', "'", 'é', 'ì', 'ð', 'í', 'i', '3', 'ù', '2', 'ô', 'q', 'æ', '\t', '%', '5', '7', '"', '\\', '#', 'v', ',', '-', 'x', 'á', '=', '\n', '\ufeff', 'û', 'ñ', '°', ']', 'm', ':', '0', '¿', 'ö', '6', 'ú', 'e', 'u', '4', '[', 'g', 'b', 'ü', 'è', 's', ';', 't', '}', '{', '<', ')', 'ý', '~', 'î', '!', '(', '_', 'l', 'p', '$', 'ò', 'ç', '8', '÷', '>', '.', '¾', '£', 'º', '×', '`', 'z', '§', '+', 'c', 'n', 'j', 'ä'}
Length of Unique characters: 109
Number of characters in data: 27956206


In [0]:
char2id = {ch:i for i, ch in enumerate(chars)}
id2char = {i:ch for i, ch in enumerate(chars)}

print ('Characters to id\n')
print (char2id)

Characters to id

{'&': 0, 'a': 1, '|': 2, 'þ': 3, 'h': 4, 'w': 5, '^': 6, 'd': 7, 'â': 8, '1': 9, 'ó': 10, 'ã': 11, 'ê': 12, 'r': 13, '@': 14, '*': 15, 'ë': 16, 'y': 17, '9': 18, 'o': 19, 'f': 20, ' ': 21, '/': 22, 'ï': 23, 'à': 24, '½': 25, 'å': 26, '¼': 27, 'k': 28, "'": 29, 'é': 30, 'ì': 31, 'ð': 32, 'í': 33, 'i': 34, '3': 35, 'ù': 36, '2': 37, 'ô': 38, 'q': 39, 'æ': 40, '\t': 41, '%': 42, '5': 43, '7': 44, '"': 45, '\\': 46, '#': 47, 'v': 48, ',': 49, '-': 50, 'x': 51, 'á': 52, '=': 53, '\n': 54, '\ufeff': 55, 'û': 56, 'ñ': 57, '°': 58, ']': 59, 'm': 60, ':': 61, '0': 62, '¿': 63, 'ö': 64, '6': 65, 'ú': 66, 'e': 67, 'u': 68, '4': 69, '[': 70, 'g': 71, 'b': 72, 'ü': 73, 'è': 74, 's': 75, ';': 76, 't': 77, '}': 78, '{': 79, '<': 80, ')': 81, 'ý': 82, '~': 83, 'î': 84, '!': 85, '(': 86, '_': 87, 'l': 88, 'p': 89, '$': 90, 'ò': 91, 'ç': 92, '8': 93, '÷': 94, '>': 95, '.': 96, '¾': 97, '£': 98, 'º': 99, '×': 100, '`': 101, 'z': 102, '§': 103, '+': 104, 'c': 105, 'n': 106, 'j': 107, 'ä'

In [0]:
# cut the text into fixed size inputs of length maxlen
maxlen = 100
sentences = []
next_chars = []

end = data_size - maxlen
for i in range(0, end, maxlen):
    sentences.append([char2id[ch] for ch in data[i : i+maxlen]])
    next_chars.append([char2id[ch] for ch in data[i+1: i+maxlen+1]])

In [0]:
print (sentences[0], len(sentences[0]), len(sentences))
print (next_chars[0], len(next_chars[0]), len(next_chars))

[55, 77, 4, 67, 21, 89, 13, 19, 107, 67, 105, 77, 21, 71, 68, 77, 67, 106, 72, 67, 13, 71, 21, 67, 72, 19, 19, 28, 21, 19, 20, 21, 5, 67, 72, 75, 77, 67, 13, 29, 75, 21, 68, 106, 1, 72, 13, 34, 7, 71, 67, 7, 21, 7, 34, 105, 77, 34, 19, 106, 1, 13, 17, 49, 21, 72, 17, 21, 48, 1, 13, 34, 19, 68, 75, 54, 54, 77, 4, 34, 75, 21, 67, 72, 19, 19, 28, 21, 34, 75, 21, 20, 19, 13, 21, 77, 4, 67, 21, 68] 100 1562
[77, 4, 67, 21, 89, 13, 19, 107, 67, 105, 77, 21, 71, 68, 77, 67, 106, 72, 67, 13, 71, 21, 67, 72, 19, 19, 28, 21, 19, 20, 21, 5, 67, 72, 75, 77, 67, 13, 29, 75, 21, 68, 106, 1, 72, 13, 34, 7, 71, 67, 7, 21, 7, 34, 105, 77, 34, 19, 106, 1, 13, 17, 49, 21, 72, 17, 21, 48, 1, 13, 34, 19, 68, 75, 54, 54, 77, 4, 34, 75, 21, 67, 72, 19, 19, 28, 21, 34, 75, 21, 20, 19, 13, 21, 77, 4, 67, 21, 68, 75] 100 1562


In [0]:
print ('Input:', ''.join(id2char[i] for i in sentences[0]))
print ('Output:', ''.join(id2char[i] for i in next_chars[0]))

Input: ﻿the project gutenberg ebook of webster's unabridged dictionary, by various

this ebook is for the u
Output: the project gutenberg ebook of webster's unabridged dictionary, by various

this ebook is for the us


In [0]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='float32')[y]

In [0]:
X = np.zeros((len(sentences), maxlen, vocab_size), dtype=np.float32)
one_hot = [to_categorical(c, num_classes=vocab_size) for i in range(len(sentences)) for c in sentences[i]]
X = np.array(one_hot).reshape(X.shape)
print (X[0], X.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (1562, 100, 109)


In [0]:
Y = np.zeros((len(next_chars), maxlen), dtype=np.float32)
Y = np.array(next_chars).reshape(Y.shape)
print (Y[0], Y.shape)

[ 77   4  67  21  89  13  19 107  67 105  77  21  71  68  77  67 106  72
  67  13  71  21  67  72  19  19  28  21  19  20  21   5  67  72  75  77
  67  13  29  75  21  68 106   1  72  13  34   7  71  67   7  21   7  34
 105  77  34  19 106   1  13  17  49  21  72  17  21  48   1  13  34  19
  68  75  54  54  77   4  34  75  21  67  72  19  19  28  21  34  75  21
  20  19  13  21  77   4  67  21  68  75] (1562, 100)


In [0]:
train_x, val_x, train_y, val_y = train_test_split(X, Y, test_size=0.2)
print ('Training:', train_x.shape, train_y.shape)
print ('Validation:', val_x.shape, val_y.shape)

Training: (1249, 100, 109) (1249, 100)
Validation: (313, 100, 109) (313, 100)


In [0]:
net = CharRNN(n_hidden=512, n_layers=2)
net.to(device)

CharRNN(
  (dropout): Dropout(p=0.5)
  (lstm): LSTM(109, 512, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=512, out_features=109, bias=True)
)

In [0]:
opt = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [0]:
train(net, train_x, train_y, val_x, val_y, opt=opt, criterion=criterion, epochs=25,
      batch_size=128, maxlen=100, clip=5, print_every=10)

Epoch: 2/25... Step: 10... Loss: 3.4775... Val Loss: 3.4398
Epoch: 3/25... Step: 20... Loss: 3.3560... Val Loss: 3.3330
Epoch: 4/25... Step: 30... Loss: 3.2230... Val Loss: 3.1962
Epoch: 5/25... Step: 40... Loss: 3.0645... Val Loss: 3.0245
Epoch: 6/25... Step: 50... Loss: 2.8395... Val Loss: 2.8186
Epoch: 7/25... Step: 60... Loss: 2.7195... Val Loss: 2.6641
Epoch: 8/25... Step: 70... Loss: 2.5964... Val Loss: 2.5909
Epoch: 9/25... Step: 80... Loss: 2.5027... Val Loss: 2.5181
Epoch: 10/25... Step: 90... Loss: 2.4404... Val Loss: 2.4613
Epoch: 12/25... Step: 100... Loss: 2.4134... Val Loss: 2.4090
Epoch: 13/25... Step: 110... Loss: 2.3840... Val Loss: 2.3708
Epoch: 14/25... Step: 120... Loss: 2.3140... Val Loss: 2.3392
Epoch: 15/25... Step: 130... Loss: 2.3620... Val Loss: 2.3083
Epoch: 16/25... Step: 140... Loss: 2.3057... Val Loss: 2.2832
Epoch: 17/25... Step: 150... Loss: 2.3228... Val Loss: 2.2521
Epoch: 18/25... Step: 160... Loss: 2.2273... Val Loss: 2.2248
Epoch: 19/25... Step: 170

In [0]:
print(sample(net, 2000, prime='defn:', top_k=5))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


defn: the abrer is the the sher strenging in torerst of borde of the a that the trene comated tomes, the calded.

abattice
ab"tho*ton, n.

defn: son bothe a sea see palle sound; seanthere absint the price. [obs.] abast.]

defn: the abord, as or abrite, to pricinatise abast.]

1. the with as a abstion.

absortation absicharin. ser abastit.]

1. (lath.

absole
deand to promint of on absound the the abotive. abstrongus of an shall abrontered with of and, ind the actare a sol the ther to tous, and in sun to a astreming in tore the ande of thar ser an to bal thatinge to abester stat on the thits abs ofe marthes of siase. abathers of absort of bor to sting the corder or seed
abated, abretintiting abord of ition abediss, an ingaling in tancity to beits or the tout, in abstion to seer the absting one with. [icp.]

defn: to a derte sores of a to seed be absint or prosing sor absouss, n. a abred.

abatias, abrace the aboter.

abserace
ab*se"ti*able, a. etym: [l. abothivieness.

defn: the abror w

## ObamaRNN

In [0]:
file_path = 'data/obama/input.txt'

with open(file_path, 'r') as f:
  
    data = f.read()
    
print ('First 200 characters of data\n')
print (data[:200])

First 200 characters of data

To Chip, Kathy, and Nancy, who graciously shared your father with a nation that loved him; to Walter's friends, colleagues, protégés, and all who considered him a hero; to the men of the Intrepid; to 


In [0]:
# unique characters in dataset and its count
chars = set(data)
vocab_size = len(set(data))
data_size = len(data)
print ('Unique characters:', chars)
print ('Length of Unique characters:', vocab_size)
print ('Number of characters in data:', data_size)

#restricting so that we can load all data onto RAM
#uncomment this line to run on all data
data = data[:156206]
data_size = len(data)

Unique characters: {'ą', '\x92', 'l', 'L', 'm', 'C', 'é', 'J', "'", 'A', 'z', 'r', ';', ')', 'f', 'v', 'c', ':', 'ֹ', 'ָ', '-', 'X', '\n', ',', '(', 'ç', '.', '”', 'ñ', 'i', 'ó', 'h', '¹', '9', '1', 'd', '8', ']', 'Q', '4', 'I', 'ד', '7', 'U', '+', 'ב', 'í', '“', '%', 'M', 'ּ', 'ï', 'ת', 'O', 'ו', 'q', '$', '¼', 'R', '>', 't', 'j', 'H', 'ł', 'B', 'ô', 'Ó', 'x', '6', 'y', 'ר', 'g', '3', 'e', '<', '&', '[', '2', 'Z', '0', 'à', 'V', 'W', 's', 'è', 'u', '5', 'S', 'á', 'E', '—', 'ה', '–', 'F', 'T', 'n', 'p', 'ę', 'D', 'P', '²', '…', '‘', '*', 'o', 'w', '`', 'Y', '’', 'K', 'k', 'G', ' ', 'b', '/', '?', 'ַ', 'N', '"', 'a', '!'}
Length of Unique characters: 121
Number of characters in data: 4224143


In [0]:
char2id = {ch:i for i, ch in enumerate(chars)}
id2char = {i:ch for i, ch in enumerate(chars)}

print ('Characters to id\n')
print (char2id)

Characters to id

{'ą': 0, '\x92': 1, 'l': 2, 'L': 3, 'm': 4, 'C': 5, 'é': 6, 'J': 7, "'": 8, 'A': 9, 'z': 10, 'r': 11, ';': 12, ')': 13, 'f': 14, 'v': 15, 'c': 16, ':': 17, 'ֹ': 18, 'ָ': 19, '-': 20, 'X': 21, '\n': 22, ',': 23, '(': 24, 'ç': 25, '.': 26, '”': 27, 'ñ': 28, 'i': 29, 'ó': 30, 'h': 31, '¹': 32, '9': 33, '1': 34, 'd': 35, '8': 36, ']': 37, 'Q': 38, '4': 39, 'I': 40, 'ד': 41, '7': 42, 'U': 43, '+': 44, 'ב': 45, 'í': 46, '“': 47, '%': 48, 'M': 49, 'ּ': 50, 'ï': 51, 'ת': 52, 'O': 53, 'ו': 54, 'q': 55, '$': 56, '¼': 57, 'R': 58, '>': 59, 't': 60, 'j': 61, 'H': 62, 'ł': 63, 'B': 64, 'ô': 65, 'Ó': 66, 'x': 67, '6': 68, 'y': 69, 'ר': 70, 'g': 71, '3': 72, 'e': 73, '<': 74, '&': 75, '[': 76, '2': 77, 'Z': 78, '0': 79, 'à': 80, 'V': 81, 'W': 82, 's': 83, 'è': 84, 'u': 85, '5': 86, 'S': 87, 'á': 88, 'E': 89, '—': 90, 'ה': 91, '–': 92, 'F': 93, 'T': 94, 'n': 95, 'p': 96, 'ę': 97, 'D': 98, 'P': 99, '²': 100, '…': 101, '‘': 102, '*': 103, 'o': 104, 'w': 105, '`': 106, 'Y': 107, '’': 10

In [0]:
# cut the text into fixed size inputs of length maxlen
maxlen = 100
sentences = []
next_chars = []

end = data_size - maxlen
for i in range(0, end, maxlen):
    sentences.append([char2id[ch] for ch in data[i : i+maxlen]])
    next_chars.append([char2id[ch] for ch in data[i+1: i+maxlen+1]])

In [0]:
print (sentences[0], len(sentences[0]), len(sentences))
print (next_chars[0], len(next_chars[0]), len(next_chars))

[94, 104, 112, 5, 31, 29, 96, 23, 112, 109, 119, 60, 31, 69, 23, 112, 119, 95, 35, 112, 117, 119, 95, 16, 69, 23, 112, 105, 31, 104, 112, 71, 11, 119, 16, 29, 104, 85, 83, 2, 69, 112, 83, 31, 119, 11, 73, 35, 112, 69, 104, 85, 11, 112, 14, 119, 60, 31, 73, 11, 112, 105, 29, 60, 31, 112, 119, 112, 95, 119, 60, 29, 104, 95, 112, 60, 31, 119, 60, 112, 2, 104, 15, 73, 35, 112, 31, 29, 4, 12, 112, 60, 104, 112, 82, 119, 2, 60, 73, 11] 100 1562
[104, 112, 5, 31, 29, 96, 23, 112, 109, 119, 60, 31, 69, 23, 112, 119, 95, 35, 112, 117, 119, 95, 16, 69, 23, 112, 105, 31, 104, 112, 71, 11, 119, 16, 29, 104, 85, 83, 2, 69, 112, 83, 31, 119, 11, 73, 35, 112, 69, 104, 85, 11, 112, 14, 119, 60, 31, 73, 11, 112, 105, 29, 60, 31, 112, 119, 112, 95, 119, 60, 29, 104, 95, 112, 60, 31, 119, 60, 112, 2, 104, 15, 73, 35, 112, 31, 29, 4, 12, 112, 60, 104, 112, 82, 119, 2, 60, 73, 11, 8] 100 1562


In [0]:
print ('Input:', ''.join(id2char[i] for i in sentences[0]))
print ('Output:', ''.join(id2char[i] for i in next_chars[0]))

Input: To Chip, Kathy, and Nancy, who graciously shared your father with a nation that loved him; to Walter
Output: o Chip, Kathy, and Nancy, who graciously shared your father with a nation that loved him; to Walter'


In [0]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='float32')[y]

In [0]:
X = np.zeros((len(sentences), maxlen, vocab_size), dtype=np.float32)
one_hot = [to_categorical(c, num_classes=vocab_size) for i in range(len(sentences)) for c in sentences[i]]
X = np.array(one_hot).reshape(X.shape)
print (X[0], X.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (1562, 100, 121)


In [0]:
Y = np.zeros((len(next_chars), maxlen), dtype=np.float32)
Y = np.array(next_chars).reshape(Y.shape)
print (Y[0], Y.shape)

[104 112   5  31  29  96  23 112 109 119  60  31  69  23 112 119  95  35
 112 117 119  95  16  69  23 112 105  31 104 112  71  11 119  16  29 104
  85  83   2  69 112  83  31 119  11  73  35 112  69 104  85  11 112  14
 119  60  31  73  11 112 105  29  60  31 112 119 112  95 119  60  29 104
  95 112  60  31 119  60 112   2 104  15  73  35 112  31  29   4  12 112
  60 104 112  82 119   2  60  73  11   8] (1562, 100)


In [0]:
train_x, val_x, train_y, val_y = train_test_split(X, Y, test_size=0.1)
print ('Training:', train_x.shape, train_y.shape)
print ('Validation:', val_x.shape, val_y.shape)

Training: (1405, 100, 121) (1405, 100)
Validation: (157, 100, 121) (157, 100)


In [0]:
net = CharRNN(n_hidden=512, n_layers=2)
net.to(device)

CharRNN(
  (dropout): Dropout(p=0.5)
  (lstm): LSTM(121, 512, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=512, out_features=121, bias=True)
)

In [0]:
opt = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [0]:
train(net, train_x, train_y, val_x, val_y, opt=opt, criterion=criterion, epochs=25,
      batch_size=128, maxlen=100, clip=5, print_every=10)

Epoch: 1/25... Step: 10... Loss: 3.3599... Val Loss: 3.3797
Epoch: 2/25... Step: 20... Loss: 3.2334... Val Loss: 3.2278
Epoch: 3/25... Step: 30... Loss: 3.1304... Val Loss: 3.1247
Epoch: 4/25... Step: 40... Loss: 3.0062... Val Loss: 2.9746
Epoch: 5/25... Step: 50... Loss: 2.8495... Val Loss: 2.8118
Epoch: 6/25... Step: 60... Loss: 2.7081... Val Loss: 2.6861
Epoch: 7/25... Step: 70... Loss: 2.6259... Val Loss: 2.6035
Epoch: 8/25... Step: 80... Loss: 2.5572... Val Loss: 2.5328
Epoch: 9/25... Step: 90... Loss: 2.4989... Val Loss: 2.4844
Epoch: 10/25... Step: 100... Loss: 2.4572... Val Loss: 2.4424
Epoch: 11/25... Step: 110... Loss: 2.4115... Val Loss: 2.3982
Epoch: 12/25... Step: 120... Loss: 2.3695... Val Loss: 2.3683
Epoch: 13/25... Step: 130... Loss: 2.3428... Val Loss: 2.3227
Epoch: 14/25... Step: 140... Loss: 2.3032... Val Loss: 2.2976
Epoch: 15/25... Step: 150... Loss: 2.2721... Val Loss: 2.2693
Epoch: 16/25... Step: 160... Loss: 2.2418... Val Loss: 2.2322
Epoch: 17/25... Step: 170.

In [0]:
print(sample(net, 2000, prime='and the question i', top_k=5))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


and the question in the workent world are and alded and is orpusiendes of our.  I went the wert thore the comsition ast whil soner wound ass that the cropetill that sumess, of to trear in sorigite and andoricates what a perichong to bela sime of the incustion or that hast to the werner a dorather taking ow and the cane then semund compeniss, and wheren this instreats. That ard war than and whe wand this in our tood..

That the sunto warested the stald couringe that will off the proce and will of there way coull cos wis porises to peation treing the promanied to to same aloure thay we came all of a bat ofer it. And that't an this the parined intores worl that world the corming anted the samo a becoramens, the same the carserest we ass are toraling oun theress all the cale to bit intery.

In that wall, and it and a tor to to chusire will has on the simus one compence to she those wish a dor on the coustions. And then wist to storman thing will here, wo live ore outhere. It wask to the wh

## TrumpRNN

In [0]:
file_path = 'data/speeches.txt'

with open(file_path, 'r') as f:
  
    data = f.read().lower()
    
print ('First 200 characters of data\n')
print (data[:100])

First 200 characters of data

﻿speech 1


...thank you so much.  that's so nice.  isn't he a great guy.  he doesn't get a fair pre


In [0]:
# unique characters in dataset and its count
chars = set(data)
vocab_size = len(set(data))
data_size = len(data)
print ('Unique characters:', chars)
print ('Length of Unique characters:', vocab_size)
print ('Number of characters in data:', data_size)

#restricting so that we can load all data onto RAM
#uncomment this line to run on all data
data = data[:496270]
data_size = len(data)

Unique characters: {'h', 'w', 'i', '!', '—', 'g', ' ', 'p', '5', '–', ',', 'n', '8', '-', '?', 'u', '’', '2', '(', ':', ')', 'k', 'o', '9', '0', '$', 'b', 'e', '/', 'y', '\n', 'v', '6', '=', 'f', '_', '.', '[', 'é', '%', '”', '&', 'q', 'j', '…', 'x', 'c', "'", ']', 'a', 's', 'l', 'd', '\ufeff', ';', 'm', 'r', '4', '“', '3', '‘', '1', 'z', '7', 't', '@', '"'}
Length of Unique characters: 67
Number of characters in data: 896270


In [0]:
char2id = {ch:i for i, ch in enumerate(chars)}
id2char = {i:ch for i, ch in enumerate(chars)}

print ('Characters to id\n')
print (char2id)

Characters to id

{'h': 0, 'w': 1, 'i': 2, '!': 3, '—': 4, 'g': 5, ' ': 6, 'p': 7, '5': 8, '–': 9, ',': 10, 'n': 11, '8': 12, '-': 13, '?': 14, 'u': 15, '’': 16, '2': 17, '(': 18, ':': 19, ')': 20, 'k': 21, 'o': 22, '9': 23, '0': 24, '$': 25, 'b': 26, 'e': 27, '/': 28, 'y': 29, '\n': 30, 'v': 31, '6': 32, '=': 33, 'f': 34, '_': 35, '.': 36, '[': 37, 'é': 38, '%': 39, '”': 40, '&': 41, 'q': 42, 'j': 43, '…': 44, 'x': 45, 'c': 46, "'": 47, ']': 48, 'a': 49, 's': 50, 'l': 51, 'd': 52, '\ufeff': 53, ';': 54, 'm': 55, 'r': 56, '4': 57, '“': 58, '3': 59, '‘': 60, '1': 61, 'z': 62, '7': 63, 't': 64, '@': 65, '"': 66}


In [0]:
# cut the text into fixed size inputs of length maxlen
maxlen = 100
sentences = []
next_chars = []

end = data_size - maxlen
for i in range(0, end, maxlen):
    sentences.append([char2id[ch] for ch in data[i : i+maxlen]])
    next_chars.append([char2id[ch] for ch in data[i+1: i+maxlen+1]])

In [0]:
print (sentences[0], len(sentences[0]), len(sentences))
print (next_chars[0], len(next_chars[0]), len(next_chars))

[53, 50, 7, 27, 27, 46, 0, 6, 61, 30, 30, 30, 36, 36, 36, 64, 0, 49, 11, 21, 6, 29, 22, 15, 6, 50, 22, 6, 55, 15, 46, 0, 36, 6, 6, 64, 0, 49, 64, 47, 50, 6, 50, 22, 6, 11, 2, 46, 27, 36, 6, 6, 2, 50, 11, 47, 64, 6, 0, 27, 6, 49, 6, 5, 56, 27, 49, 64, 6, 5, 15, 29, 36, 6, 6, 0, 27, 6, 52, 22, 27, 50, 11, 47, 64, 6, 5, 27, 64, 6, 49, 6, 34, 49, 2, 56, 6, 7, 56, 27] 100 4962
[50, 7, 27, 27, 46, 0, 6, 61, 30, 30, 30, 36, 36, 36, 64, 0, 49, 11, 21, 6, 29, 22, 15, 6, 50, 22, 6, 55, 15, 46, 0, 36, 6, 6, 64, 0, 49, 64, 47, 50, 6, 50, 22, 6, 11, 2, 46, 27, 36, 6, 6, 2, 50, 11, 47, 64, 6, 0, 27, 6, 49, 6, 5, 56, 27, 49, 64, 6, 5, 15, 29, 36, 6, 6, 0, 27, 6, 52, 22, 27, 50, 11, 47, 64, 6, 5, 27, 64, 6, 49, 6, 34, 49, 2, 56, 6, 7, 56, 27, 50] 100 4962


In [0]:
print ('Input:', ''.join(id2char[i] for i in sentences[0]))
print ('Output:', ''.join(id2char[i] for i in next_chars[0]))

Input: ﻿speech 1


...thank you so much.  that's so nice.  isn't he a great guy.  he doesn't get a fair pre
Output: speech 1


...thank you so much.  that's so nice.  isn't he a great guy.  he doesn't get a fair pres


In [0]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='float32')[y]

In [0]:
X = np.zeros((len(sentences), maxlen, vocab_size), dtype=np.float32)
one_hot = [to_categorical(c, num_classes=vocab_size) for i in range(len(sentences)) for c in sentences[i]]
X = np.array(one_hot).reshape(X.shape)
print (X[0], X.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (4962, 100, 67)


In [0]:
Y = np.zeros((len(next_chars), maxlen), dtype=np.float32)
Y = np.array(next_chars).reshape(Y.shape)
print (Y[0], Y.shape)

[50  7 27 27 46  0  6 61 30 30 30 36 36 36 64  0 49 11 21  6 29 22 15  6
 50 22  6 55 15 46  0 36  6  6 64  0 49 64 47 50  6 50 22  6 11  2 46 27
 36  6  6  2 50 11 47 64  6  0 27  6 49  6  5 56 27 49 64  6  5 15 29 36
  6  6  0 27  6 52 22 27 50 11 47 64  6  5 27 64  6 49  6 34 49  2 56  6
  7 56 27 50] (4962, 100)


In [0]:
train_x, val_x, train_y, val_y = train_test_split(X, Y, test_size=0.2)
print ('Training:', train_x.shape, train_y.shape)
print ('Validation:', val_x.shape, val_y.shape)

Training: (3969, 100, 67) (3969, 100)
Validation: (993, 100, 67) (993, 100)


In [0]:
net = CharRNN(n_hidden=512, n_layers=2)
net.to(device)

CharRNN(
  (dropout): Dropout(p=0.5)
  (lstm): LSTM(67, 512, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=512, out_features=67, bias=True)
)

In [0]:
opt = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [0]:
train(net, train_x, train_y, val_x, val_y, opt=opt, criterion=criterion, epochs=25,
      batch_size=128, maxlen=100, clip=5, print_every=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch: 1/25... Step: 10... Loss: 3.2287... Val Loss: 3.1822
Epoch: 1/25... Step: 20... Loss: 3.0906... Val Loss: 3.0858
Epoch: 1/25... Step: 30... Loss: 2.9882... Val Loss: 2.9710
Epoch: 2/25... Step: 40... Loss: 2.8379... Val Loss: 2.8247
Epoch: 2/25... Step: 50... Loss: 2.6758... Val Loss: 2.6598
Epoch: 2/25... Step: 60... Loss: 2.5457... Val Loss: 2.5285
Epoch: 3/25... Step: 70... Loss: 2.4881... Val Loss: 2.4531
Epoch: 3/25... Step: 80... Loss: 2.3684... Val Loss: 2.3902
Epoch: 3/25... Step: 90... Loss: 2.3475... Val Loss: 2.3347
Epoch: 4/25... Step: 100... Loss: 2.3043... Val Loss: 2.2940
Epoch: 4/25... Step: 110... Loss: 2.2788... Val Loss: 2.2524
Epoch: 4/25... Step: 120... Loss: 2.2056... Val Loss: 2.2205
Epoch: 5/25... Step: 130... Loss: 2.2091... Val Loss: 2.1832
Epoch: 5/25... Step: 140... Loss: 2.1355... Val Loss: 2.1538
Epoch: 5/25... Step: 150... Loss: 2.1225... Val Loss: 2.1275
Epoch: 6/25... Step: 160... Loss: 2.0748... Val Loss: 2.0937
Epoch: 6/25... Step: 170... Loss:

In [0]:
print(sample(net, 2000, prime='fake news', top_k=5))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


fake news immigration is trump to be a sprescibe to talk and we have got to see what i’m doing is they statting the wert ago.
and they’re not comerang and tell you, and it’s not a creat startest this is a border of mane is almest so incredible again. we had a toter change, and to say the world where it is to get and they don’t know.
we have to go ialo. i see the some if i were with the thill believe when i was showling, and they are going to be some of them to do this.


and if you can did that he don’t win. it’s a contratelley that’s has bigger. but here, we’re going to stopy isis backed a letter and the well. i’ll be the special i think that we have a seecial parts of middle east.






and i went in with mover of television, is a courle people to do what they’re going to be saying it’s not.






thank you very group. it’s a love all oversite, we have to start the wart our military is a besa fight of them. i’m not going to do a lot of people. i didn’t thoug this and we’re going to b

## AnnaRNN

In [0]:
file_path = 'data/anna.txt'

with open(file_path, 'r') as f:
  
    data = f.read().lower()
    
print ('First 200 characters of data\n')
print (data[:200])

First 200 characters of data

chapter 1


happy families are all alike; every unhappy family is unhappy in its own
way.

everything was in confusion in the oblonskys' house. the wife had
discovered that the husband was carrying on


In [0]:
# unique characters in dataset and its count
chars = set(data)
vocab_size = len(set(data))
data_size = len(data)
print ('Unique characters:', chars)
print ('Length of Unique characters:', vocab_size)
print ('Number of characters in data:', data_size)

#restricting so that we can load all data onto RAM
#uncomment this line to run on all data
data = data[:585223]
data_size = len(data)

Unique characters: {';', 'a', '`', 'q', '5', '4', 'v', '0', '-', 'd', 'c', 'm', '2', ',', 'b', '6', 'r', ')', 'u', '1', 'l', 'x', 'i', 'f', 's', '?', ':', 'y', '(', '_', ' ', 'z', '8', 'g', "'", 'e', '9', '.', '!', '"', '3', 'n', '\n', 'o', 'h', 'p', 't', 'j', '7', 'w', 'k'}
Length of Unique characters: 51
Number of characters in data: 985223


In [0]:
char2id = {ch:i for i, ch in enumerate(chars)}
id2char = {i:ch for i, ch in enumerate(chars)}

print ('Characters to id\n')
print (char2id)

Characters to id

{';': 0, 'a': 1, '`': 2, 'q': 3, '5': 4, '4': 5, 'v': 6, '0': 7, '-': 8, 'd': 9, 'c': 10, 'm': 11, '2': 12, ',': 13, 'b': 14, '6': 15, 'r': 16, ')': 17, 'u': 18, '1': 19, 'l': 20, 'x': 21, 'i': 22, 'f': 23, 's': 24, '?': 25, ':': 26, 'y': 27, '(': 28, '_': 29, ' ': 30, 'z': 31, '8': 32, 'g': 33, "'": 34, 'e': 35, '9': 36, '.': 37, '!': 38, '"': 39, '3': 40, 'n': 41, '\n': 42, 'o': 43, 'h': 44, 'p': 45, 't': 46, 'j': 47, '7': 48, 'w': 49, 'k': 50}


In [0]:
# cut the text into fixed size inputs of length maxlen
maxlen = 100
sentences = []
next_chars = []

end = data_size - maxlen
for i in range(0, end, maxlen):
    sentences.append([char2id[ch] for ch in data[i : i+maxlen]])
    next_chars.append([char2id[ch] for ch in data[i+1: i+maxlen+1]])

In [0]:
print (sentences[0], len(sentences[0]), len(sentences))
print (next_chars[0], len(next_chars[0]), len(next_chars))

[10, 44, 1, 45, 46, 35, 16, 30, 19, 42, 42, 42, 44, 1, 45, 45, 27, 30, 23, 1, 11, 22, 20, 22, 35, 24, 30, 1, 16, 35, 30, 1, 20, 20, 30, 1, 20, 22, 50, 35, 0, 30, 35, 6, 35, 16, 27, 30, 18, 41, 44, 1, 45, 45, 27, 30, 23, 1, 11, 22, 20, 27, 30, 22, 24, 30, 18, 41, 44, 1, 45, 45, 27, 30, 22, 41, 30, 22, 46, 24, 30, 43, 49, 41, 42, 49, 1, 27, 37, 42, 42, 35, 6, 35, 16, 27, 46, 44, 22, 41] 100 5852
[44, 1, 45, 46, 35, 16, 30, 19, 42, 42, 42, 44, 1, 45, 45, 27, 30, 23, 1, 11, 22, 20, 22, 35, 24, 30, 1, 16, 35, 30, 1, 20, 20, 30, 1, 20, 22, 50, 35, 0, 30, 35, 6, 35, 16, 27, 30, 18, 41, 44, 1, 45, 45, 27, 30, 23, 1, 11, 22, 20, 27, 30, 22, 24, 30, 18, 41, 44, 1, 45, 45, 27, 30, 22, 41, 30, 22, 46, 24, 30, 43, 49, 41, 42, 49, 1, 27, 37, 42, 42, 35, 6, 35, 16, 27, 46, 44, 22, 41, 33] 100 5852


In [0]:
print ('Input:', ''.join(id2char[i] for i in sentences[0]))
print ('Output:', ''.join(id2char[i] for i in next_chars[0]))

Input: chapter 1


happy families are all alike; every unhappy family is unhappy in its own
way.

everythin
Output: hapter 1


happy families are all alike; every unhappy family is unhappy in its own
way.

everything


In [0]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='float32')[y]

In [0]:
X = np.zeros((len(sentences), maxlen, vocab_size), dtype=np.float32)
one_hot = [to_categorical(c, num_classes=vocab_size) for i in range(len(sentences)) for c in sentences[i]]
X = np.array(one_hot).reshape(X.shape)
print (X[0], X.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (5852, 100, 51)


In [0]:
Y = np.zeros((len(next_chars), maxlen), dtype=np.float32)
Y = np.array(next_chars).reshape(Y.shape)
print (Y[0], Y.shape)

[44  1 45 46 35 16 30 19 42 42 42 44  1 45 45 27 30 23  1 11 22 20 22 35
 24 30  1 16 35 30  1 20 20 30  1 20 22 50 35  0 30 35  6 35 16 27 30 18
 41 44  1 45 45 27 30 23  1 11 22 20 27 30 22 24 30 18 41 44  1 45 45 27
 30 22 41 30 22 46 24 30 43 49 41 42 49  1 27 37 42 42 35  6 35 16 27 46
 44 22 41 33] (5852, 100)


In [0]:
train_x, val_x, train_y, val_y = train_test_split(X, Y, test_size=0.2)
print ('Training:', train_x.shape, train_y.shape)
print ('Validation:', val_x.shape, val_y.shape)

Training: (4681, 100, 51) (4681, 100)
Validation: (1171, 100, 51) (1171, 100)


In [0]:
net = CharRNN(n_hidden=512, n_layers=2)
net.to(device)

CharRNN(
  (dropout): Dropout(p=0.5)
  (lstm): LSTM(51, 512, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=512, out_features=51, bias=True)
)

In [0]:
opt = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [0]:
train(net, train_x, train_y, val_x, val_y, opt=opt, criterion=criterion, epochs=25,
      batch_size=128, maxlen=100, clip=5, print_every=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch: 1/25... Step: 10... Loss: 3.1530... Val Loss: 3.1431
Epoch: 1/25... Step: 20... Loss: 3.0492... Val Loss: 3.0508
Epoch: 1/25... Step: 30... Loss: 2.9280... Val Loss: 2.9294
Epoch: 2/25... Step: 40... Loss: 2.7354... Val Loss: 2.7629
Epoch: 2/25... Step: 50... Loss: 2.6036... Val Loss: 2.6110
Epoch: 2/25... Step: 60... Loss: 2.5185... Val Loss: 2.5182
Epoch: 2/25... Step: 70... Loss: 2.4501... Val Loss: 2.4556
Epoch: 3/25... Step: 80... Loss: 2.3872... Val Loss: 2.4131
Epoch: 3/25... Step: 90... Loss: 2.3642... Val Loss: 2.3764
Epoch: 3/25... Step: 100... Loss: 2.3246... Val Loss: 2.3413
Epoch: 4/25... Step: 110... Loss: 2.3332... Val Loss: 2.3079
Epoch: 4/25... Step: 120... Loss: 2.2664... Val Loss: 2.2792
Epoch: 4/25... Step: 130... Loss: 2.2599... Val Loss: 2.2588
Epoch: 4/25... Step: 140... Loss: 2.2387... Val Loss: 2.2247
Epoch: 5/25... Step: 150... Loss: 2.2071... Val Loss: 2.2088
Epoch: 5/25... Step: 160... Loss: 2.1550... Val Loss: 2.1776
Epoch: 5/25... Step: 170... Loss:

In [0]:
print(sample(net, 2000, prime='anna', top_k=5))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


anna, his chear, and was at his brother, she sat down the forts of a force that his cales was straight for his beginning to her sore, a sittle of
it, but at some tell the paits.

"you want it," said stepan arkadyevitch.

"what i had be noter with the morning. the conversation was stord out him a fore an her atiently in the
some and she had
been delighted up the door, stepan arkadyevitch was some at her.

"oh, then well i down the pression of her hand out the canter with the
dinner as all her has should and saw that her sat her hand, and his hands. "as they way, too," alexey alexandrovitch was she was
askandary the came to asking him a lift a charg, and still he would have so that the masche stand in his was strought. the same
on the success of to the princess had to been to the sof the should
hand to the pland was a find of compare and hulding
towards the conter he had been all at the set that when the sare horses of a shame stirlly was simply stoom the talker had been down that she wa